In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(dotenv_path="../keys/.env")

True

In [2]:
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

In [3]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

In [4]:
openAI_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# List of PDF file paths
pdf_dir = "../data/pdfs"
pdf_files = [os.path.join(pdf_dir, file) for file in os.listdir(pdf_dir) if file.endswith(".pdf")]

# URLs to load
urls = [
    "https://www.math-datascience.nat.fau.de/im-studium/masterstudiengaenge/master-data-science/",
    "https://www.fau.eu/studiengang/data-science-bsc/",
    "https://www.fau.eu/studiengang/data-science-msc/",
]

# Initialize an empty list to hold all document splits
all_doc_splits = []

# Define the text splitter with a chunk size of 1000 characters and overlap of 200
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, 
    chunk_overlap=200)

### Process PDF files ###
for pdf_file in pdf_files:
    # Load the PDF
    pdf_loader = PyPDFLoader(pdf_file)
    documents = pdf_loader.load()
    
    # Split the documents into chunks
    doc_splits = text_splitter.split_documents(documents)
    
    # Add the splits to the overall list
    all_doc_splits.extend(doc_splits)

### Process Web URLs ###
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split the web documents into chunks
web_doc_splits = text_splitter.split_documents(docs_list)

# Add the web document splits to the overall list
all_doc_splits.extend(web_doc_splits)

# Now, all_doc_splits contains the splits from both the PDF files and the web documents.

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
print(len(all_doc_splits))

108


In [7]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=all_doc_splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [8]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines and give only questions not anything else, not even statements like Here are five search queries:. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | ChatGroq(model_name="llama-3.1-70b-versatile") 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [10]:
questions = [
    # 'Will the language of instruction of my Bachelor’s degree suffice to demonstrate my proficiency in English for admission in Msc Data Science at FAU Erlangen?',
    # 'Is an APS certificate mandatory for Indian students during the application process?',
    # 'How long does it typically take to receive a decision regarding my application to the MSc Data Science program?',
    # 'Am I eligible to apply for the study course program with a degree from a technical university or a university of applied sciences?',
    # 'Can I apply for the study course program with a degree obtained through a dual study program?',
    # 'What could be the reasons for receiving a rejection for my application?',
    # 'Is it possible to defer my admission to a future semester?',
    # 'Where can I find accommodation while studying at FAU?',
    # 'Can I switch my major subject after being admitted?',
    # 'Is it allowed to combine modules from different application subjects?',
    # 'How can I transfer ECTS credits from one module group to another?',
    # 'Can modules from my major subject count towards my minor subject requirements?',
    # 'Can my werkstudent experience be used to earn ECTS as a technical qualification?',
    # 'Can I participate in an examination without prior registration?',
    # 'How do I register for a module’s examination in Campo within the correct specialisation area?',
    # 'If I fail an examination on the first attempt, is it mandatory to attend the next attempt?',
    # 'Is it possible to withdraw from an examination after registering?',
    # 'What should I do if I cannot attend an examination due to illness?',
    # 'What are the consequences of failing an examination three times?',
    # 'Can I retake a completed module to improve my grades?',
    # 'Is it possible to improve my grades by completing additional modules?',
    # 'How many ECTS credits are required before I can start writing my Master’s thesis?',
    # 'How can I find a topic for my Master’s thesis?',
    # 'Who is eligible to supervise my Master’s thesis?',
    # 'Can my Master’s thesis supervisor be from a different department?',
    # 'Is it possible to complete my Master’s thesis while working in a company?',
    # 'Is it necessary to have a university supervisor for a company-based thesis?',
    # 'What is the process for registering my Master’s thesis?',
    # 'What steps should I take after completing my Master’s thesis?',

    # 'What are the admission requirements of M.Sc. Data Science at FAU Erlangen?',
    # 'How can I apply for M.Sc. Data Science at FAU Erlangen?',
    # 'Is accommodation available at FAU Erlangen?',
    # 'How can I finance my studies at FAU Erlangen?',
    # 'What are the next steps if I get an admission at FAU Erlangen?',
    # 'What is the duration of M.Sc. Data Science degree program at FAU Erlangen?',
    # 'What is the teaching language of M.Sc. Data Science at FAU Erlangen?',
    # 'What is the structure of M.Sc. Data Science at FAU Erlangen?',
    # 'Is German required for M.Sc. Data Science at FAU Erlangen?',
    # 'What is the duration of B.Sc. Data Science degree program at FAU Erlangen?',
    # 'What is the teaching language of B.Sc. Data Science at FAU Erlangen?',
    'What is the structure of B.Sc. Data Science FAU Erlangen?',
    # 'What are the admission requirements of B.Sc. Data Science FAU Erlangen?'

]

In [11]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

In [12]:
answers = []
for question in questions:
    retrieval_chain = generate_queries | retriever.map() | get_unique_union
    docs = retrieval_chain.invoke({"question":question})

    from operator import itemgetter
    from langchain_openai import ChatOpenAI
    from langchain_core.runnables import RunnablePassthrough

    # RAG
    template = """Answer the following question in detail and based on this context:

    {context}

    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)

    final_rag_chain = (
        {"context": retrieval_chain, 
        "question": itemgetter("question")} 
        | prompt
        | ChatGroq(model_name="llama-3.1-70b-versatile")
        | StrOutputParser()
    )

    ans = final_rag_chain.invoke({"question":question})
    answers.append(ans)

c:\Users\Ekaansh_Khosla\anaconda3\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [13]:
answers

["The B.Sc. Data Science program at FAU Erlangen-Nürnberg is structured into several consecutive levels. \n\n1. The first level focuses on foundational courses in mathematics and computer science, which provide a solid understanding of the basics of these subjects. \n2. The second level builds upon the foundational knowledge by offering advanced courses in mathematics and computer science. \n3. The third level allows students to specialize in a specific application subject, such as physics, business informatics, biology, or medical technology. \n4. The fourth level consists of a bachelor's thesis, which requires students to apply their knowledge and skills to a real-world problem.\n\nAdditionally, the program offers a semester abroad option, which allows students to spend one semester at another European university, such as through the ERASMUS program. The standard period of study for the bachelor's degree program is six semesters.\n\nThe program is designed to provide students with a 